<div style="text-align: right"> Mark Hendricks </div>

<left>FINM 37500 - Fixed Income Derivatives</left> 
<br>
<left>Winter 2024</left>

<h2><center> Homework #3- Modeling Volatility, Pricing w/ BDT, Midcurve Swaptions </center></h2>

<center>Due: Wednesday, Feb 28 at 6PM</center>

<h3><span style="color:#00008B">Name - Nick Lewis</span></h3>
<h3><span style="color:#00008B">Email - nicklewis16@uchicago.edu</span></h3>

# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

In [3]:
import numpy as np
import pandas as pd
import datetime
import warnings
from scipy.stats import norm
from scipy.optimize import fsolve

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)
import warnings
warnings.filterwarnings('ignore')


import sys
sys.path.insert(0, '../cmds')
from my_cmds import *

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

In [4]:
swaption_vol = pd.read_excel(f'../data/swaption_vol_data.xlsx')

freqcurve = 4
RELATIVE_STRIKE = 0
N = 100

SWAP_TYPE = 'SOFR'
QUOTE_STYLE = 'black'
expry = 1
tenor = 4
DATE = '2024-02-20'

isPayer = True
freqswap = 4
swaption_vol

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

In [5]:
FILEDATE = '2024-02-20'
FILEIN = f'../data/cap_quotes_{FILEDATE}'
BB_COMPOUND = 1 #compounding of quoted SOFR swaps
freqcap = 4
DATE = '2024-02-20'

In [6]:
## QUOTES TO CURVES
SHEET = 'sofr'
sofrdata_raw = pd.read_excel(f'{FILEIN}.xlsx', sheet_name=SHEET).set_index('date')
sofrdata_raw.columns = sofrdata_raw.loc['maturity'] 
sofrdata_raw.drop(index=['maturity'],inplace=True)
sofrdata_raw.index = pd.to_datetime(sofrdata_raw.index)

sofrdata_raw.columns.name ='maturity'
sofrdata_raw /= 100
sofrdata_raw = sofrdata_raw.T.drop_duplicates().T
sofrdata = sofrdata_raw.copy()
sofrdata = compound_rate(sofrdata,BB_COMPOUND,freqcap)
sofrcurves = interp_curves(sofrdata, dt = 1/freqcap, date=DATE, interp_method='cubicspline').rename(columns={'interp':'swap rates'})
sofrquotes = sofrdata.loc[DATE,:].rename('quotes')

sofrcurves['spot rates'] = bootstrap_discounts_clean(sofrcurves[['swap rates']],compounding=4,key='swap rates')
sofrcurves['discounts'] = ratecurve_to_discountcurve(sofrcurves['spot rates'],n_compound=freqcap)
sofrcurves['forwards'] = ratecurve_to_forwardcurve(sofrcurves['spot rates'],n_compound=freqcap)

In [7]:
SHEET = 'cap'
capdata = pd.read_excel(f'{FILEIN}.xlsx', sheet_name=SHEET).set_index('date')
capdata.columns = capdata.loc['maturity'] 
capdata.drop(index=['maturity'],inplace=True)
capdata.index = pd.to_datetime(capdata.index)
capdata.columns = (freqcap * capdata.columns.values).round(0)/freqcap
capdata.columns.name ='maturity'
capdata = capdata.T.drop_duplicates().T
capquotes = capdata.loc[DATE,:].to_frame()
capquotes.columns = ['normal']
capquotes[DATE] = capquotes['normal'] / sofrcurves['forwards'] / 100**2
LIN_EXTRAP = True
FRONT_RATIO = .65

capcurves = interp_curves(capquotes[[DATE]].T,dt=1/freqcap, date=DATE,interp_method='cubicspline').rename(columns={'interp':'flat vols'})

if LIN_EXTRAP:
    fix_short = capcurves.loc[:1,'flat vols']
    fix_short.iloc[:-1] = np.nan
    fix_short.iloc[0] = capcurves.loc[1,'flat vols'] * FRONT_RATIO
    capcurves.loc[:1,'flat vols'] = fix_short.interpolate(limit_direction='both', fill_value = 'extrapolate')

# drop extrapolated value at first period as there is no caplet for the first period.
capcurves['flat vols'].iloc[0] = np.nan
capcurves = flat_to_forward_vol_rev(capcurves['flat vols'],sofrcurves['swap rates'],sofrcurves['forwards'],sofrcurves['discounts'],freq=4)
curves = pd.concat([sofrcurves.drop(columns=['quotes']), capcurves.drop(columns=['cap prices'])],axis=1)
curves.index.name = 'tenor'
curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247
1.50,0.046235,0.046141,0.933499,0.039030,0.292615,0.380930
1.75,0.045059,0.044939,0.924774,0.037738,0.315878,0.388953
2.00,0.044133,0.043994,0.916212,0.037382,0.331443,0.386643
2.25,0.043173,0.043011,0.908230,0.035151,0.340919,0.376247


Functions Used:

## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

In [8]:
Topt = expry
Tswap = Topt + tenor

fwdrate = curves['forwards'].loc[Tswap]

fwdswap = calc_fwdswaprate(curves['discounts'], Topt, Tswap, freqswap = freqswap)

print(f"Forward swap rate: {fwdswap:.4%}")

Forward swap rate: 3.6722%



## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.


In [9]:
voldata = pd.read_excel(f'../data/swaption_vol_data.xlsx')
volquote = voldata.query(f'model == "{QUOTE_STYLE}"').query(f'reference == "{SWAP_TYPE}"').query(f'date == "{DATE}"')
idx = (volquote['expiration'] == expry) & (volquote['tenor'] == tenor)
volquote = volquote.loc[idx]
volquote.index = ['implied vol']
strikerange = np.array(volquote.columns[-9:].tolist())
vols = volquote[strikerange]
vols /= 100
strikes = fwdswap + strikerange/100/100
idstrike = np.where(strikerange == RELATIVE_STRIKE)[0][0]
idstrikeATM = np.where(strikerange == 0)[0][0]

if QUOTE_STYLE =='normal':
    vols /= 100 * fwdrate

capvol = curves.loc[Topt, 'fwd vols']

strikeATM = strikes[idstrikeATM]
volATM = vols.iloc[0,idstrikeATM]

period_fwd = curves.index.get_loc(Topt)
period_swap = curves.index.get_loc(Tswap) + 1
step = round(freqcurve/ freqswap)

discount = curves['discounts'].iloc[period_fwd + step :period_swap:step].sum() / freqswap
black_quotes = vols.copy()
black_quotes.loc['price'] = N * blacks_formula(Topt, vols, strikes, fwdswap, discount, isCall = isPayer)[0]
black_quotes.loc['strike'] = strikes
black_quotes.style.format('{:.4f}')

,-200,-100,-50,-25,0,25,50,100,200
implied vol,0.5454,0.4037,0.3594,0.3423,0.3283,0.3171,0.3086,0.2983,0.2954
price,7.2031,4.0653,2.7398,2.1749,1.6874,1.2812,0.9556,0.5133,0.1473
strike,0.0167,0.0267,0.0317,0.0342,0.0367,0.0392,0.0417,0.0467,0.0567



## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [10]:
curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247
1.50,0.046235,0.046141,0.933499,0.039030,0.292615,0.380930
1.75,0.045059,0.044939,0.924774,0.037738,0.315878,0.388953
2.00,0.044133,0.043994,0.916212,0.037382,0.331443,0.386643
2.25,0.043173,0.043011,0.908230,0.035151,0.340919,0.376247


In [11]:
def calculate_black_quotes(curves, freqcurve, freqswap, expiries, tenors, volATM, strikeATM, fwdswap, discount, isPayer, N = 100):
    """
    Calculate the Black's formula prices for a set of swaptions.
    
    Parameters:
    - curves
    - freqcurve
    - freqswap
    - expiries (list): List of swaption expiries
    - tenors (list): List of swaption tenors
    - fwdswaps (np.ndarray): Array to store the calculated forward swap rates
    - volATM (float): ATM implied volatility
    - strikeATM (float): ATM strike
    - fwdswap (float): Forward swap rate
    - discount (float): Discount factor
    - isPayer (bool): True if swaption is a payer swaption, False otherwise
    
    Returns:
    - black_quotes_alt (pd.DataFrame): DataFrame containing the calculated Black's formula prices
    """
    black_quotes_alt = pd.DataFrame(dtype=float, columns=['expiry', 'tenor', 'price'])
    fwdswaps = np.full(len(expiries), np.nan)

    for i in range(len(fwdswaps)):
        fwdswaps[i] = calc_fwdswaprate(curves['discounts'], expiries[i], tenors[i] + expiries[i], freqswap=freqswap)

        period0 = curves.index.get_loc(expiries[i])
        period1 = curves.index.get_loc(expiries[i] + tenors[i]) + 1
        step_i = round(freqcurve / freqswap)

        discount_i = curves['discounts'].iloc[period0 + step_i: period1: step_i].sum() / freqswap

        black_quotes_alt.loc[i, ['expiry', 'tenor']] = [expiries[i], tenors[i]]
        black_quotes_alt.loc[i, 'price'] = N * blacks_formula(expiries[i], volATM, strikeATM, fwdswap, discount_i, isCall=isPayer)
    
    return black_quotes_alt

def highlight_row(row):
    if row.name == 3:
        return ['background-color: yellow'] * len(row)
    else:
        return[''] * len(row)


expiries = [.25, 2, 1, 1]
tenors = [4,4,2, 4]
black_quotes_alt = calculate_black_quotes(curves, freqcurve, freqswap, expiries, tenors, volATM, strikeATM, fwdswap, curves['discounts'],  isPayer)
black_quotes_alt.style.apply(highlight_row, axis =1).format({'expiry': '{:.2f}', 'tenor': '{:.2f}', 'price': '{:.4f}'})

,expiry,tenor,price
0,0.25,4.00,0.8705
1,2.00,4.00,2.2914
2,1.00,2.00,0.8740
3,1.00,4.00,1.6874


***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

In [42]:
DATE = '2024-02-20'
curves = pd.read_excel(f'../data/cap_curves_{DATE}.xlsx', sheet_name = f'rate curves {DATE}', index_col= 'tenor')

FREQTREE = 4
dt = 1/FREQTREE

newgrid = np.arange(dt,curves.index[-1]+dt,dt).round(6)
curvesint = pd.DataFrame(dtype=float, columns=curves.columns, index=newgrid)
cols_interp = ['swap rates','flat vols','fwd vols']

# interpolate
curvesint = pd.concat([curvesint,curves],axis=0)
curvesint = curvesint.interpolate(method='cubicspline',fill_value='extrapolate',limit_direction='both')
curvesint = curvesint.loc[newgrid,:]
curvesint = curvesint[~curvesint.index.duplicated()].sort_index()

# compute derived curves
curvesint['discounts'] = ratecurve_to_discountcurve(curvesint['swap rates'],n_compound=FREQTREE)
curvesint['forwards'] = ratecurve_to_forwardcurve(curvesint['swap rates'],n_compound=FREQTREE)

# forward vol is derived but can't extract curve--interpolate and adjust front to first flat vol
id0 = curves['flat vols'].dropna().index[0]
curvesint.loc[:id0,['flat vols','fwd vols']] = curves.loc[id0,'flat vols']

In [43]:
quotes = curves['discounts']
swaprate = calc_swaprate(quotes, Tswap, freqswap=freqswap)
fwdswap = calc_fwdswaprate(quotes, Topt, Tswap, freqswap=freqswap)
fwdrate = curves['forwards'][Topt]

sigmas = curvesint['fwd vols']
sigmas.iloc[0] = sigmas.iloc[1]
sigmas.index = sigmas.index.to_numpy().round(6)

theta, ratetree = estimate_theta(sigmas.loc[:Tswap],100*curvesint.loc[:Tswap,'discounts'])

In [46]:
format_bintree(ratetree.loc[:,:Tswap], style = '{:.4%}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,5.1874%,5.4736%,5.6989%,5.9217%,5.9410%,6.4159%,7.0858%,8.0900%,8.8669%,10.5371%,12.6050%,15.1273%,17.2605%,20.2390%,24.9329%,29.9666%,34.6657%,41.0519%,50.3879%,61.1453%
1,,4.6363%,4.8271%,5.0158%,5.0321%,5.4344%,6.0019%,6.8524%,7.5105%,8.9252%,10.6767%,12.8132%,14.6201%,17.1430%,21.1188%,25.3825%,29.3627%,34.7720%,42.6798%,51.7916%
2,,,3.9102%,4.0631%,4.0763%,4.4022%,4.8619%,5.5509%,6.0840%,7.2299%,8.6488%,10.3795%,11.8431%,13.8868%,17.1075%,20.5613%,23.7856%,28.1674%,34.5732%,41.9543%
3,,,,3.1507%,3.1610%,3.4137%,3.7701%,4.3044%,4.7178%,5.6064%,6.7067%,8.0488%,9.1838%,10.7685%,13.2660%,15.9443%,18.4445%,21.8424%,26.8098%,32.5335%
4,,,,,2.2026%,2.3787%,2.6271%,2.9994%,3.2874%,3.9066%,4.6733%,5.6084%,6.3993%,7.5036%,9.2438%,11.1101%,12.8523%,15.2199%,18.6812%,22.6695%
5,,,,,,1.6252%,1.7949%,2.0492%,2.2460%,2.6691%,3.1929%,3.8318%,4.3722%,5.1266%,6.3156%,7.5907%,8.7810%,10.3986%,12.7635%,15.4884%
6,,,,,,,1.2165%,1.3889%,1.5223%,1.8090%,2.1640%,2.5971%,2.9633%,3.4747%,4.2805%,5.1447%,5.9515%,7.0478%,8.6507%,10.4975%
7,,,,,,,,0.9435%,1.0341%,1.2289%,1.4701%,1.7643%,2.0131%,2.3605%,2.9079%,3.4950%,4.0430%,4.7878%,5.8767%,7.1313%
8,,,,,,,,,0.7099%,0.8436%,1.0091%,1.2111%,1.3818%,1.6203%,1.9961%,2.3991%,2.7753%,3.2865%,4.0339%,4.8952%


In [47]:
annual_ratetree = compound_rate(ratetree, None, 1)
format_bintree(annual_ratetree.loc[:,:Tswap], style = '{:.4%}')

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,5.3243%,5.6262%,5.8644%,6.1005%,6.1210%,6.6262%,7.3429%,8.4263%,9.2719%,11.1122%,13.4339%,16.3315%,18.8397%,22.4326%,28.3164%,34.9409%,41.4332%,50.7600%,65.5129%,84.3108%
1,,4.7455%,4.9455%,5.1437%,5.1609%,5.5848%,6.1857%,7.0927%,7.7997%,9.3356%,11.2676%,13.6703%,15.7429%,18.7001%,23.5144%,28.8946%,34.1284%,41.5836%,53.2343%,67.8526%
2,,,3.9877%,4.1468%,4.1606%,4.5006%,4.9820%,5.7078%,6.2728%,7.4977%,9.0338%,10.9373%,12.5730%,14.8973%,18.6579%,22.8278%,26.8526%,32.5347%,41.3024%,52.1266%
3,,,,3.2009%,3.2115%,3.4726%,3.8421%,4.3984%,4.8309%,5.7666%,6.9367%,8.3815%,9.6187%,11.3697%,14.1861%,17.2857%,20.2551%,24.4114%,30.7475%,38.4494%
4,,,,,2.2270%,2.4072%,2.6619%,3.0448%,3.3420%,3.9839%,4.7842%,5.7687%,6.6085%,7.7923%,9.6845%,11.7508%,13.7147%,16.4392%,20.5401%,25.4448%
5,,,,,,1.6385%,1.8111%,2.0704%,2.2714%,2.7050%,3.2444%,3.9062%,4.4692%,5.2603%,6.5193%,7.8862%,9.1781%,10.9585%,13.6138%,16.7523%
6,,,,,,,1.2239%,1.3986%,1.5339%,1.8255%,2.1876%,2.6311%,3.0076%,3.5357%,4.3734%,5.2794%,6.1321%,7.3021%,9.0359%,11.0683%
7,,,,,,,,0.9480%,1.0395%,1.2365%,1.4810%,1.7799%,2.0335%,2.3885%,2.9506%,3.5568%,4.1259%,4.9043%,6.0528%,7.3917%
8,,,,,,,,,0.7124%,0.8471%,1.0142%,1.2184%,1.3914%,1.6335%,2.0161%,2.4281%,2.8141%,3.3411%,4.1164%,5.0170%


## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

In [51]:
freqcpn = freqswap
strike = strikes[idstrike]
cpn = strike
FREQTREE = 4
dt = 1/FREQTREE
N = 100

wrapper_bond = lambda r: payoff_bond(r, 1/FREQTREE, facevalue=N * (1+cpn/freqcpn))
cftree = construct_bond_cftree(Tswap, FREQTREE, cpn = cpn, cpn_freq=freqcpn, face=N)
bondtree = bintree_pricing(payoff=wrapper_bond, ratetree=ratetree, cftree=cftree)



    
format_bintree(bondtree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,98.68,96.54,94.30,91.93,89.42,86.71,83.92,81.10,78.33,75.58,73.07,70.90,69.20,67.96,67.39,67.88,69.65,72.84,78.07,86.61
1,,101.55,99.61,97.55,95.35,92.96,90.47,87.91,85.35,82.75,80.29,78.08,76.22,74.71,73.75,73.72,74.84,77.25,81.51,88.66
2,,,104.03,102.25,100.37,98.32,96.16,93.93,91.68,89.35,87.10,85.02,83.18,81.57,80.38,79.93,80.43,82.00,85.18,90.87
3,,,,106.01,104.39,102.64,100.81,98.91,96.98,94.98,93.02,91.16,89.48,87.95,86.70,86.01,86.03,86.87,88.95,93.03
4,,,,,107.47,105.96,104.40,102.78,101.14,99.45,97.78,96.18,94.71,93.34,92.16,91.40,91.15,91.48,92.71,95.36
5,,,,,,108.32,106.95,105.55,104.13,102.67,101.23,99.84,98.54,97.30,96.21,95.40,94.96,94.93,95.52,97.09
6,,,,,,,108.74,107.48,106.22,104.93,103.65,102.41,101.23,100.10,99.07,98.24,97.67,97.37,97.51,98.30
7,,,,,,,,108.82,107.66,106.48,105.32,104.18,103.08,102.03,101.04,100.19,99.52,99.05,98.87,99.13
8,,,,,,,,,108.66,107.56,106.47,105.40,104.36,103.35,102.38,101.52,100.79,100.18,99.78,99.69


## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

In [52]:
swap_tree = 100 - bondtree
swap_tree.style.format('{:.2f}', na_rep = '')

time,0.000000,0.250000,0.500000,0.750000,1.000000,1.250000,1.500000,1.750000,2.000000,2.250000,2.500000,2.750000,3.000000,3.250000,3.500000,3.750000,4.000000,4.250000,4.500000,4.750000
state,,,,,,,,,,,,,,,,,,,,
0,1.32,3.46,5.70,8.07,10.58,13.29,16.08,18.90,21.67,24.42,26.93,29.10,30.80,32.04,32.61,32.12,30.35,27.16,21.93,13.39
1,,-1.55,0.39,2.45,4.65,7.04,9.53,12.09,14.65,17.25,19.71,21.92,23.78,25.29,26.25,26.28,25.16,22.75,18.49,11.34
2,,,-4.03,-2.25,-0.37,1.68,3.84,6.07,8.32,10.65,12.90,14.98,16.82,18.43,19.62,20.07,19.57,18.00,14.82,9.13
3,,,,-6.01,-4.39,-2.64,-0.81,1.09,3.02,5.02,6.98,8.84,10.52,12.05,13.30,13.99,13.97,13.13,11.05,6.97
4,,,,,-7.47,-5.96,-4.40,-2.78,-1.14,0.55,2.22,3.82,5.29,6.66,7.84,8.60,8.85,8.52,7.29,4.64
5,,,,,,-8.32,-6.95,-5.55,-4.13,-2.67,-1.23,0.16,1.46,2.70,3.79,4.60,5.04,5.07,4.48,2.91
6,,,,,,,-8.74,-7.48,-6.22,-4.93,-3.65,-2.41,-1.23,-0.10,0.93,1.76,2.33,2.63,2.49,1.70
7,,,,,,,,-8.82,-7.66,-6.48,-5.32,-4.18,-3.08,-2.03,-1.04,-0.19,0.48,0.95,1.13,0.87
8,,,,,,,,,-8.66,-7.56,-6.47,-5.40,-4.36,-3.35,-2.38,-1.52,-0.79,-0.18,0.22,0.31


## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

In [54]:
T = 1
tsteps = int(T/dt)+1
swaptreeT = swap_tree.iloc[:tsteps,:tsteps]
cts_ratetreeT = ratetree.iloc[:tsteps,:tsteps]
payoff = lambda v: np.maximum(v,0)

swaption_tree = bintree_pricing(payoff = payoff,  ratetree=ratetree, undertree= swaptreeT)
format_bintree(swaption_tree)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.73,2.94,4.83,7.50,10.58
1,,0.56,1.13,2.30,4.65
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00


## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

In [55]:
print('Blacks Formula: ', black_quotes_alt.loc[3]['price'])
print('Binomial tree: ', swaption_tree.loc[0,0])
print('Price difference (binom - blacks): ',  swaption_tree.loc[0,0] - black_quotes_alt.loc[3]['price'])

Blacks Formula:  1.6873830102383778
Binomial tree:  1.7284411360224925
Price difference (binom - blacks):  0.04105812578411472


Binomial tree estimates a larger price compared to blacks formula by .64.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

In [56]:
swaption_tree2 = bintree_pricing(payoff = payoff,  ratetree=ratetree, undertree= swaptreeT, style = 'american')
format_bintree(swaption_tree2)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,2.00,3.46,5.70,8.07,10.58
1,,0.60,1.21,2.45,4.65
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00


***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y $\rightarrow$ 2y $\rightarrow$ 2y.

* expires in 1 yr
* underlying swap starts 2 years after that expiration (in 3 years from today)
* swap runs for 2 years, (until 5 years from today.)

In [59]:
Topt=1
Tmid=2
Ttenor=2

In [60]:
if freqswap==FREQTREE:
    swaprate_val = strike
    payoff = lambda r: payoff_swap(r,swaprate_val,freqswap,ispayer=isPayer)
    cftree, refratetree = construct_swap_cftree(ratetree,swaprate_val,freqswap=freqswap,T=Tswap,freq=FREQTREE,ispayer=isPayer)

    # midcurve swap
    cftree_mid = cftree.copy()
    cftree_mid.loc[:,:(Topt+Tmid-1/freqswap)] *= 0

    swaptree_mid = bintree_pricing(payoff=payoff, ratetree=ratetree, undertree= refratetree, cftree=cftree_mid, cfdelay=True)

In [61]:
payoff_swaption = lambda s: np.maximum(s,0)
swaptiontree_mid = bintree_pricing(payoff=payoff_swaption,ratetree=ratetree.loc[:,:Topt].dropna(how='all'),undertree=swaptree_mid.loc[:,:Topt].dropna(how='all'))
format_bintree(swaptiontree_mid)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.86,1.48,2.46,3.88,5.63
1,,0.27,0.55,1.11,2.24
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00


## 3.2

Price the **american** midcurve swaption 1y $\rightarrow$ 2y $\rightarrow$ 2y.

In [62]:
swaptiontree_mid_american = bintree_pricing(style='american',payoff=payoff_swaption,ratetree=ratetree.loc[:,:Topt].dropna(how='all'),undertree=swaptree_mid.loc[:,:Topt].dropna(how='all'))
format_bintree(swaptiontree_mid_american)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.86,1.48,2.46,3.88,5.63
1,,0.27,0.55,1.11,2.24
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00


***